**merge.py**
+ reads in NOAA converted coordinates and merge with accident, road, grade, curve files


In [4]:
import os
import pandas as pd
import sqlite3

### functions

In [1]:
def detect_files(directory, keyword):
    """
    detect files in specified directory with specified keyword

    input
    -----
    directory : string
        dir to search

    keyword : string
        keyword to look for

    output
    -----
    sorted list of file names

    test
    -----
        (1) if output has larger than length;
    """

    file_list = []
    for file in os.listdir(directory):
        if keyword in file:
            file_list.append(file)
    
    return sorted(file_list)


In [2]:
def read_files(directory, keyword):
    """
    read files with specified keyword

    input
    -----
    directory : string
        directory to read files from

    keyword : string
        keyword to search for

    output
    -----
    output_dic : dic
        dictionary of datasets

    test
    -----
        (1) output_dic should have length 5, for 2013 - 2017;
        (2) keyword should not be empty;
    """
    output_dic = {}
    file_list = detect_files(directory, keyword)
    for yr in range(2013, 2018):
        output_dic[yr] = pd.read_csv(os.path.join(directory, file_list[yr-2013]))
    return output_dic


### process

In [5]:
crash = read_files("../hsis-csv", 'acc')
veh = read_files("../hsis-csv", 'veh')
peds = read_files("../hsis-csv", 'peds')
occ = read_files("../hsis-csv", 'occ')

road = read_files("../hsis-csv", 'road')
curv = read_files("../hsis-csv", 'curv')
grad = read_files("../hsis-csv", 'grad')


veh_drop = []
peds_drop = []
occ_drop = []
road_drop = ['COUNTY', 'DISTRICT', 'FUNC_CLS', 'RD_TYPE', 'RTE_NBR', 'rodwycls']
curv_drop = []
grad_drop = []

/Users/shuyiyin/anaconda/envs/syin/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/shuyiyin/anaconda/envs/syin/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
yr = 2013
# veh, needs aggregation
veh_cnt = veh[yr]['CASENO'].value_counts().sort_index()
veh_cnt = veh_cnt.to_frame().reset_index()
veh_cnt.columns = ['CASENO', 'veh_cnt']

# peds, needs aggregation
peds_cnt = peds[yr]['CASENO'].value_counts().sort_index()
peds_cnt = peds_cnt.to_frame().reset_index()
peds_cnt.columns = ['CASENO', 'peds_cnt']

# occs, need aggregation
occ_cnt = occ[yr]['CASENO'].value_counts().sort_index()
occ_cnt = occ_cnt.to_frame().reset_index()
occ_cnt.columns = ['CASENO', 'occ_cnt']

# inner or left? left I think
acc_this_yr = crash[yr].merge(veh_cnt, on='CASENO', how='left')
acc_this_yr = acc_this_yr.merge(peds_cnt, on='CASENO', how='left')
acc_this_yr = acc_this_yr.merge(occ_cnt, on='CASENO', how='left')

In [9]:
# road
road_this_yr = road[yr].drop(road_drop, axis=1)
conn = sqlite3.connect(":memory:")
acc_this_yr.to_sql("crash", conn, index=False)
road_this_yr.to_sql("road", conn, index=False)

query =  "SELECT * FROM crash, road WHERE crash.rd_inv = road.ROAD_INV AND crash.milepost >= road.BEGMP AND crash.milepost <= road.ENDMP"
records = pd.read_sql_query(query, conn)

## remove duplicates randomly
records = records.sample(frac=1).drop_duplicates(subset='CASENO').sort_index()
# remove duplicate connecting keys
records = records.drop(['ROAD_INV', 'BEGMP', 'ENDMP'], axis=1)

In [10]:
# curve
curv_this_yr = curv[yr].drop(curv_drop, axis=1)
conn = sqlite3.connect(":memory:")
records.to_sql("records", conn, index=False)
curv_this_yr.to_sql("curv", conn, index=False)

query = "SELECT * FROM records LEFT JOIN curv ON records.rd_inv = curv.curv_inv AND records.milepost >= curv.begmp AND records.milepost <= curv.endmp"
records = pd.read_sql_query(query, conn)

## remove duplicates and drop useless attributes
records = records.sample(frac=1).drop_duplicates(subset='CASENO').sort_index()
records = records.drop(['curv_inv', 'begmp', 'endmp'], axis=1)

## fill NaN curvature with 0
records = records.fillna(value={'deg_curv': 0})

In [11]:
# grad
grad_this_yr = grad[yr].drop(grad_drop, axis=1)
conn = sqlite3.connect(":memory:")
records.to_sql("records", conn, index=False)
grad_this_yr.to_sql("grad", conn, index=False)

query = "SELECT * FROM records LEFT JOIN grad ON records.rd_inv = grad.grad_inv AND records.milepost >= grad.begmp AND records.milepost <= grad.endmp"

records = pd.read_sql_query(query, conn)

records = records.sample(frac=1).drop_duplicates(subset='CASENO').sort_index()
records = records.drop(['grad_inv', 'begmp', 'endmp'], axis=1)
records = records.fillna(value={'pct_grad': 0})

In [13]:
for i in range(records.shape[0]):
    if not isinstance(records.iloc[i, -2], str):
        records.iloc[i, -2] = '0'

In [19]:
records.head(10)

,rd_inv,milepost,rodwycls,CASENO,ACCYR,RTE_NBR,RD_TYPE,DISTRICT,COUNTY,rur_urb,...,TRLR_LG2,TRLR_WD1,TRLR_WD2,AADT,trkpcts,mvmt,DIR_CURV,deg_curv,dir_grad,pct_grad
0,002,0.00,99,2013016932,2013,2,None,1,31,U,...,0.0,0,0,6593.0,3.34,0.10,None,0.00,-,2.19
1,002,0.08,2,2013013626,2013,2,None,1,31,U,...,0.0,0,0,6593.0,3.34,0.17,None,0.00,-,2.19
2,002,0.18,1,201337343,2013,2,None,1,31,U,...,0.0,0,0,23213.0,3.75,0.17,L,43.74,-,3.87
3,002,0.19,1,201328176,2013,2,None,1,31,U,...,0.0,0,0,21132.0,0.00,0.15,L,43.74,-,0.19
4,002,0.19,1,2013010812,2013,2,None,1,31,U,...,0.0,0,0,23213.0,3.75,0.17,L,43.74,-,0.19
5,002,0.23,1,2013020310,2013,2,None,1,31,U,...,0.0,0,0,21132.0,0.00,0.08,R,17.74,-,0.19
6,002,0.31,2,201333763,2013,2,None,1,31,U,...,0.0,0,0,21132.0,0.00,0.08,None,0.00,+,2.76
7,002,0.35,2,2013025035,2013,2,None,1,31,U,...,0.0,0,0,21132.0,0.00,0.23,L,0.93,+,2.76
8,002,0.35,2,201329568,2013,2,None,1,31,U,...,0.0,0,0,21132.0,0.00,0.23,L,0.93,+,2.76
9,002,0.38,1,201337339,2013,2,None,1,31,U,...,0.0,0,0,21132.0,0.00,0.23,L,0.93,+,5.00
